<a href="https://colab.research.google.com/github/natsumeyuya/portfolio/blob/master/%E5%96%B6%E6%A5%AD%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0%E3%83%AA%E3%82%B9%E3%83%88%E6%94%B9%E8%A8%82%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#インポート
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
import time
#抽出したデータの格納先
df = pd.DataFrame()
#クラス
class Restaurant_list:
  def __init__(self,base_url,district):
    self.district = district
    self.base_url = base_url
  #URL取得
  def get_url(self):
    num = 1
    url_list = []
    append = url_list.append
    while num < 60:
      url = self.base_url + str(num)
      append(url)
      html = requests.get(url)
      soup = BeautifulSoup(html.content, 'lxml')
      print(soup.find(class_ = 'c-page-count__num').text)
      if soup.find(class_ = 'c-page-count__num').text == "0":
        break
      num += 1
      time.sleep(1)
    return url_list

  #データの抽出
  def get_data(self,url_list):
    shop_list = []
    genre_info = []
    area_info = []
    rate_list = []
    call_list = []
    rest_list = []
    homepage_list = []
    number = str(len(url_list))

    for target_url in url_list[:len(url_list)-1]:
      html = requests.get(target_url)
      soup = BeautifulSoup(html.content, 'lxml')
      print("残り" + number + "回")
      number = str(int(number) - 1)
      #店舗名
      raw_data = soup.find_all('a', class_='list-rst__rst-name-target cpy-rst-name')
      shop_list += [name.text for name in raw_data]
      shop_url_list = [shop_url.get('href') for shop_url in raw_data]

      #ジャンル
      genre_info += [genre.text.rstrip().split("/")[1] for genre in soup.find_all('div', class_='list-rst__area-genre cpy-area-genre')]

      #エリア
      area_info += [genre.text.strip().split("/")[0] for genre in soup.find_all('div', class_='list-rst__area-genre cpy-area-genre')]

      #評価点&ホームページ＆電話番号&定休日
      append1 = rate_list.append
      append2 = rest_list.append
      append3 = homepage_list.append
      append4 = call_list.append
      for shop_url in shop_url_list:
        response = requests.get(shop_url)
        all_data = BeautifulSoup(response.content, 'lxml')
        shop_header = all_data.find(class_ = 'rstdtl-header')
        shop_data = all_data.find(class_ = 'rstinfo-table')
        try:
          rate = shop_header.find(class_="rdheader-rating__score-val-dtl")
          append1(float(rate.text))
        except:
          append1('-')
        try:
          rest = shop_header.find(class_ ='rdheader-subinfo__closed-text')
          append2(rest.text.strip())
        except:
          append2('-')
        try:
          homepage = shop_data.find('p', class_ = 'homepage').find('a',class_ = 'c-link-arrow')
          append3(homepage.get('href'))
        except:
          append3('-')
        try:
          call = shop_data.find(class_ = "rstinfo-table__tel-num")
          append4(call.text)
        except:
          append4('-')
        time.sleep(1)
    #データフレームに格納
    df["name"] = shop_list
    df["genre"] = genre_info
    df["area"] = area_info
    df["rate"] = rate_list
    df["call"] = call_list
    df["rest"] = rest_list
    df["homepage"] = homepage_list

    #欲しいデータの抽出
    data = df[df['homepage'].str.contains('http:')]
    data = data.reset_index()
    data = data.drop('index',axis=1)
    return data

  #Excelへのデータの書き込み
  def write_xlsx(self,data):
    with pd.ExcelWriter('/content/drive/MyDrive/営業/営業リスト.xlsx',mode='a') as writer:
      data.to_excel(writer, sheet_name= f"{self.district}")

  #プログラム実行
  def work(self):
    print(f'{self.district}')
    url_list = self.get_url()
    data =  self.get_data(url_list)
    self.write_xlsx(data)
    

In [ ]:
oosu =  Restaurant_list("https://tabelog.com/aichi/A2301/A230105/R1737/rstLst/","大須観音")
yabatyou_1 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R10228/rstLst/RC/","矢場町_レストラン")
yabatyou_2 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R10228/rstLst/MC/","矢場町_ラーメン")
yabatyou_3 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R10228/rstLst/cafe/","矢場町_カフェ")
yabatyou_4 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R10228/rstLst/SC/","矢場町_パン")
yabatyou_5 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R10228/rstLst/bar/","矢場町_バー")
sakae_1 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R4222/rstLst/RC/","栄＿レストラン")
sakae_2 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R4222/rstLst/MC/","栄＿ラーメン")
sakae_3 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R4222/rstLst/cafe/","栄＿カフェ")
sakae_4 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R4222/rstLst/SC/","栄＿パン")
sakae_5 = Restaurant_list("https://tabelog.com/aichi/A2301/A230103/R4222/rstLst/bar/","栄＿バー")
nagoya_1 = Restaurant_list("https://tabelog.com/aichi/A2301/A230101/R7155/rstLst/RC/","名古屋＿レストラン")
nagoya_2 = Restaurant_list("https://tabelog.com/aichi/A2301/A230101/R7155/rstLst/MC/","名古屋＿ラーメン")
nagoya_3 = Restaurant_list("https://tabelog.com/aichi/A2301/A230101/R7155/rstLst/cafe/","名古屋＿カフェ")
nagoya_4 = Restaurant_list("https://tabelog.com/aichi/A2301/A230101/R7155/rstLst/SC/","名古屋＿パン")
nagoya_5 = Restaurant_list("https://tabelog.com/aichi/A2301/A230101/R7155/rstLst/bar/","名古屋＿バー")
kamimaezu =  Restaurant_list("https://tabelog.com/aichi/A2301/A230105/R2722/rstLst/","上前津")
higasi = Restaurant_list("https://tabelog.com/aichi/C23102/rstLst/","東区")
syouwa = Restaurant_list("https://tabelog.com/aichi/C23107/rstLst/","昭和区")
tikusa = Restaurant_list("https://tabelog.com/aichi/C23101/rstLst/","千種区")
tennpaku = Restaurant_list("https://tabelog.com/aichi/C23116/rstLst/","天白区")